In [1]:
import pandas as pd
import mysql.connector
from config import DB, USR, PWD

In [2]:
con = mysql.connector.connect(database=DB, user=USR, password=PWD)
cur = con.cursor()

**Вопросы копия в доп**

In [9]:
dop = pd.read_sql("SELECT id, question_list, question_num FROM questions WHERE question_letter = 'доп' AND question_text IS NULL", con=con)

In [10]:
dop

,id,question_list,question_num
0,1710,IVa,19
1,1711,XXIIa,14
2,1712,IIIa,9
3,1713,XVI,1
4,1714,XXIa,12
...,...,...,...
229,2003,XVIII,14
230,2005,Ia,10
231,2006,IVa,7
232,2010,XIX,3


In [8]:
questions = pd.read_sql("SELECT id, question_list, question_num, question_text, question_theme FROM questions WHERE question_letter = ''", con=con)

In [11]:
questions

,id,question_list,question_num,question_text,question_theme
0,1,Ia,1,Где и как выбирали невесту?,Выбор невесты
1,2,Ia,2,Сватовство,Сватовство
2,8,Ia,3,Рукобитье.,Рукобитье
3,12,Ia,4,Смотрины,Смотрины
4,15,Ia,5,Что происходит в доме невесты от смотрин до ве...,Подготовка невесты к свадьбе
...,...,...,...,...,...
543,1693,XXVII,19,Иуда,Иуда
544,1702,XXVII,20,Конец света.,Конец света
545,1840,XI,0,None,None
546,1994,XXII,19,None,None


In [18]:
dop.merge(questions, on=["question_list", "question_num"])["id_x"].value_counts()

1935    3
2011    1
1796    1
1806    1
1805    1
       ..
1896    1
1895    1
1894    1
1893    1
1710    1
Name: id_x, Length: 210, dtype: int64

In [22]:
new = dop.merge(questions, on=["question_list", "question_num"])
new = new[~new["id_x"].isin([1935])]

In [27]:
cur.executemany("UPDATE questions SET question_text = %s, question_theme = %s WHERE id = %s",
            new[["question_text", "question_theme", "id_x"]].values.tolist())
con.commit()

In [28]:
dop = pd.read_sql("SELECT id, question_list, question_num FROM questions WHERE question_letter = 'доп' AND question_text IS NULL", con=con)

In [30]:
dop.shape

(22, 3)

In [35]:
questions = pd.read_sql("SELECT question_list, question_list_name FROM q_list_name", con=con)

In [38]:
new = dop.merge(questions)

In [40]:
cur.executemany("UPDATE questions SET question_text = %s, question_theme = %s WHERE id = %s",
            new[["question_list_name", "question_list_name", "id"]].values.tolist())
con.commit()

## Старые данные

In [3]:
df = pd.read_csv("data/done/tblCards.csv")
df.head(2)

,текст,аннотация,ключевые слова,термины,программа,вопрос,село,год,информант,информант2,...,информант4,собиратель,собиратель2,собиратель3,собиратель4,номер,zvuk,zvuk2,photo,photo2
0,"[Шёл разговор о пьющей женщине из Карпиловки, ...",NaN,"Водка, Плата, Питье, Девственность, Женщина, П...",NaN,XXIa,5 б,"Брянская обл., Злынковский р-н, Спиридонова Буда",2017,А,Г,...,NaN,МЭГ,НВП,СЕ,ФЕЕ,1,NaN,NaN,NaN,NaN
1,[Разговор о кусачей собаке:] [Г:] А раз её\ э\...,NaN,"Первый, Прозвище, Имя, Еврей, Народ, Свой-чужой",NaN,XXIa,5 б,"Брянская обл., Злынковский р-н, Спиридонова Буда",2017,А,Г,...,NaN,МЭГ,НВП,СЕ,ФЕЕ,2,NaN,NaN,NaN,NaN


In [127]:
df = pd.read_sql_query("SELECT * FROM informators", con=con)

In [4]:
import numpy as np

In [128]:
drop_ids = [450, 452, 457, 455, 458, 461, 463, 464, 462, 460, 454, 433, 465, 466, 467, 447, 446, 448, 449, 451, 453, 456, 436, 458, 459, 443, 465]

# 258 Южа, Лукино
# 261 Южа, Южа

In [129]:
df = df[~df["id"].isin(drop_ids)].astype("object").where(pd.notnull(df),None)
cur.executemany("INSERT INTO informators2 VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", df.values.tolist())
con.commit()

In [234]:
df

,id_text,id_informator
30,61,124
31,61,125
36,66,125
37,66,124
124,141,87
...,...,...
10936,7290,463
10937,7291,463
10938,7292,463
10939,7293,463


In [355]:
df = pd.read_csv("data/done/tblCards.csv")
df.head(2)

,текст,аннотация,ключевые слова,термины,программа,вопрос,село,год,информант,информант2,...,информант4,собиратель,собиратель2,собиратель3,собиратель4,номер,zvuk,zvuk2,photo,photo2
0,"[Шёл разговор о пьющей женщине из Карпиловки, ...",NaN,"Водка, Плата, Питье, Девственность, Женщина, П...",NaN,XXIa,5 б,"Брянская обл., Злынковский р-н, Спиридонова Буда",2017,А,Г,...,NaN,МЭГ,НВП,СЕ,ФЕЕ,1,NaN,NaN,NaN,NaN
1,[Разговор о кусачей собаке:] [Г:] А раз её\ э\...,NaN,"Первый, Прозвище, Имя, Еврей, Народ, Свой-чужой",NaN,XXIa,5 б,"Брянская обл., Злынковский р-н, Спиридонова Буда",2017,А,Г,...,NaN,МЭГ,НВП,СЕ,ФЕЕ,2,NaN,NaN,NaN,NaN


In [341]:
geo_map = {
    "Глубочица": "Себеж",
    "Спиридонова Буда": "Рогов",
    "Рогов": "Карпиловка",
    "Себеж": "Псков",
}

name_map = {
    "C": "С",
    "ЧВФ": "ЧФВ",
    "ЮС": "СЮ",
    "ВЛ": "ЛВ"
}

In [342]:
from nltk.tokenize import wordpunct_tokenize

In [343]:
def clean(text):
    text = wordpunct_tokenize(text)
    return "".join(i for i in text if i.isalpha()).lower()

In [344]:
K = 300
cur.execute("SELECT raw_text, year, id FROM texts")
text_map = dict()

text_seen = set()

for i, year, j in cur.fetchall():
    if i:
        tclean = clean(i)[:K]
        if (tclean, year) not in text_seen:
            text_map[(tclean, year)] = j
            text_seen.add((tclean, year))
        else:
            print(text_map[(tclean, year)], j, end=";")
            del text_map[(tclean, year)]

2332 3194;2333 3195;2334 3196;2335 3197;2336 3198;2337 3199;2338 3200;2339 3201;2209 3639;2210 3640;2211 3641;2212 3642;2213 3643;2214 3644;2215 3645;2216 3646;3988 3989;4902 4903;5265 5266;6927 6928;6944 6945;4484 7082;7199 7200;7240 7241;

In [345]:
result = []

for i in range(df.shape[0]):
    row = df.iloc[i]
    text_id = (clean(row["текст"])[:K], row["год"])
    if text_id not in text_map:
        print("========", row["номер"], row["текст"][:100])
        continue
    selo = row["село"].split(",")[-1].strip()
    names = []
    for name in ["информант", "информант2", "информант3", "информант4"]:
        if type(row[name]) == str:
            if "," in row[name]:
                names.extend(row[name].split(","))
            elif row[name].strip("???"):
                names.append(row[name])
#     print("||", row["номер"], names, end="\t")
    for name in names:
        name = name.strip("?").strip()
        if name in name_map:
            name = name_map[name]
        if name not in counter:
            print(row["номер"], name, selo, names)
            continue
        if counter[name] == 1:
            infor = code_village[name]
        elif selo in code_village[name]:
            infor = code_village[name][selo]
        elif geo_map[selo] in code_village[name]:
            infor = code_village[name][geo_map[selo]]    
        else:
            print(row["номер"], name, selo, code_village[name])
            continue
#         print(text_map[text_id], infor, end="--")
        result.append((text_map[text_id], infor))
#     print()

194 КВТ Спиридонова Буда ['КВТ']
208 ХВН Спиридонова Буда ['АВС', 'ХВН', 'БВ', 'АС']
4547 ГРВ Эпимахово ['ГРВ', 'ГАВ']
590  Азаричи ['ЛЕМ', ' ???']
1600 СРС Азаричи ['СРС', 'ЛПС']
2026 ЛС Малые Щербиничи ['ЛС']
======== 2213 [Рассказывает, что у отца была мельница.] [А мельница осталась?] Не, нема\, во\т, э\то, где\ шко\ла 
======== 2214 [Песни из тетрадки НФЛ, которая она слышала по радио и исполняла в клубе: «Поговори со мною, мама», 
======== 2215 [Напевает по записям некоторые песни, которые когда-то пели в клубе.] [А у вас есть записи местных п
======== 2216 [Красный угол в доме информантки имеет небольшие шторки, на одной из икон есть белая пелена из круже
======== 2217 [Говорит, что держит огород в 25 соток, раньше держала коров, теперь хочет завести уток.] Дак что ты
======== 2218 [Соб: Нас интересует что-нибудь про народную медицину, как кто-нибудь к знахарю ходил…] А, к зна\хар
======== 2219 [До этого говорили о том, что умели заговарить грыжу.] [А помните, как заговаривали, 

In [347]:
cur.executemany("INSERT INTO t_i (id_text, id_informator) VALUES (%s, %s)", result)
con.commit()

In [349]:
replace_inf = {
    433: 258,
    436: 261,
    443: 282,
    446: 424,
    447: 425,
    448: 426,
    449: 427,
    450: 428,
    451: 429,
    452: 430,
    453: 431,
    454: 432,
    455: 258,
    456: 434,
    457: 435,
    458: 261,
    459: 437,
    460: 438,
    461: 439,
    462: 440,
    463: 441,
    464: 442,
    465: 282,
    466: 444,
    467: 445
}

cur.execute("SELECT id_text FROM t_i")
seen = {i[0] for i in cur.fetchall()}

df = pd.read_sql_query("SELECT id_text, id_informator FROM t_i_archived JOIN texts ON id_text = texts.id WHERE leader = 'ЮМК'", con=con)
df = df[~df["id_text"].isin(seen)]

df

,id_text,id_informator
0,4090,287
1,4090,288
2,4090,289
3,4091,287
4,4091,288
...,...,...
2387,7290,463
2388,7291,463
2389,7292,463
2390,7293,463


In [350]:
df['id_informator'] = df['id_informator'].apply(lambda x: replace_inf.get(x, x))

In [351]:
cur.executemany("INSERT INTO t_i (id_text, id_informator) VALUES (%s, %s)", df.values.tolist())
con.commit()

In [362]:
K = 300
cur.execute("SELECT raw_text, year, id FROM texts WHERE leader = 'АБМ'")
text_map = dict()

text_seen = set()

for i, year, j in cur.fetchall():
    if i:
        tclean = clean(i)[:K]
        l = len(clean(i))
        if (tclean, year) not in text_seen:
            text_map[(tclean, year)] = (j, l)
            text_seen.add((tclean, year))
        else:
            print(text_map[(tclean, year)][0], j, end=";")
            del text_map[(tclean, year)]

2332 3194;2333 3195;2334 3196;2335 3197;2336 3198;2337 3199;2338 3200;2339 3201;2209 3639;2210 3640;2211 3641;2212 3642;2213 3643;2214 3644;2215 3645;2216 3646;3988 3989;

In [367]:
result = []

for i in range(df.shape[0]):
    row = df.iloc[i]
    text_id = (clean(row["текст"])[:K], row["год"])
    text_len = len(clean(row["текст"]))
    if text_id not in text_map:
#         print("========", row["номер"], row["текст"][:100])
        continue
    else:
        result.append(tuple([*text_map[text_id], text_len]))

In [368]:
result = [tuple([*row, abs(row[1] - row[2])]) for row in result if abs(row[1] - row[2]) > 0]
result = sorted(result, key=lambda x: x[-1], reverse=True)
result = [i[0] for i in result]

In [371]:
",".join([str(i) for i in result])

'3145,6233,2217,2402,6767,6232,6019,2825,5459,6125,2628,1353,2408,6693,2269,6474,850,2822,1358,837,458,848,6853,6662,5878,2222,3428,459,3068,471,3673,6314,6234,303,6030,963,5752,2781,5709,1684,6168,1950,3599,5605,1914,150,1363,5460,1819,2611,5977,3600,1729,1060,467,5799,6017,6562,147,5788,3874,922,6066,2218,3157,1155,2604,1299'

### собиратели

In [5]:
import numpy as np
from nltk.tokenize import wordpunct_tokenize

In [6]:
df = pd.read_csv("data/done/tblCards.csv")
df.head(2)

,текст,аннотация,ключевые слова,термины,программа,вопрос,село,год,информант,информант2,...,информант4,собиратель,собиратель2,собиратель3,собиратель4,номер,zvuk,zvuk2,photo,photo2
0,"[Шёл разговор о пьющей женщине из Карпиловки, ...",NaN,"Водка, Плата, Питье, Девственность, Женщина, П...",NaN,XXIa,5 б,"Брянская обл., Злынковский р-н, Спиридонова Буда",2017,А,Г,...,NaN,МЭГ,НВП,СЕ,ФЕЕ,1,NaN,NaN,NaN,NaN
1,[Разговор о кусачей собаке:] [Г:] А раз её\ э\...,NaN,"Первый, Прозвище, Имя, Еврей, Народ, Свой-чужой",NaN,XXIa,5 б,"Брянская обл., Злынковский р-н, Спиридонова Буда",2017,А,Г,...,NaN,МЭГ,НВП,СЕ,ФЕЕ,2,NaN,NaN,NaN,NaN


In [7]:
def clean(text):
    text = wordpunct_tokenize(text)
    return "".join(i for i in text if i.isalpha()).lower()

In [14]:
K = 300
cur.execute("SELECT raw_text, year, id FROM texts")
text_map = dict()

text_seen = set()

for i, year, j in cur.fetchall():
    if i:
        tclean = clean(i)[:K]
        if (tclean, year) not in text_seen:
            text_map[(tclean, year)] = j
            text_seen.add((tclean, year))
        else:
            print(text_map[(tclean, year)], j, end=";")
            del text_map[(tclean, year)]

2332 3194;2333 3195;2334 3196;2335 3197;2336 3198;2337 3199;2338 3200;2339 3201;2209 3639;2210 3640;2211 3641;2212 3642;2213 3643;2214 3644;2215 3645;2216 3646;3988 3989;4902 4903;5265 5266;6927 6928;6944 6945;4484 7082;7199 7200;7240 7241;

In [14]:
df.columns

Index(['текст', 'аннотация', 'ключевые слова', 'термины', 'программа',
       'вопрос', 'село', 'год', 'информант', 'информант2', 'информант3',
       'информант4', 'собиратель', 'собиратель2', 'собиратель3', 'собиратель4',
       'номер', 'zvuk', 'zvuk2', 'photo', 'photo2'],
      dtype='object')

In [15]:
collectors_dict = dict(pd.read_sql_query("SELECT code, id FROM collectors", con=con).values)

In [23]:
# sorted(collectors_dict.items())

In [16]:
name_map = {
    35: 58
}

In [10]:
# ЛКВ
# МВА
# ИАВ

In [17]:
result = []

for i in range(df.shape[0]):
    row = df.iloc[i]
    text_id = (clean(row["текст"])[:K], row["год"])
    if text_id not in text_map:
#         print("========", row["номер"], row["текст"][:100])
        continue
    selo = row["село"].split(",")[-1].strip()
    names = []
    for name in ["собиратель", "собиратель2", "собиратель3", "собиратель4"]:
        if type(row[name]) == str:
            if "," in row[name]:
                names.extend(row[name].split(","))
            elif row[name].strip("???"):
                names.append(row[name])
    if set([n for n in names if n not in collectors_dict]):
        print("||", row["номер"], set([n for n in names if n not in collectors_dict]))
    for n in names:
        result.append((text_map[text_id], collectors_dict[n]))

In [19]:
cur.executemany("INSERT INTO t_c (id_text, id_collector) VALUES (%s, %s)", result)
con.commit()

In [22]:
cur.execute("SELECT id_text, id_collector FROM t_c_archived WHERE id_text IN (SELECT id FROM texts WHERE leader = 'ЮМК')")

data = cur.fetchall()

In [24]:
cur.executemany("INSERT INTO t_c (id_text, id_collector) VALUES (%s, %s)", data)
con.commit()

In [25]:
no_text = []

for i in range(df.shape[0]):
    row = df.iloc[i]
    text_id = (clean(row["текст"])[:K], row["год"])
    if text_id not in text_map:
        no_text.append(row["номер"])

In [33]:
# no_text

In [27]:
df2 = df[df["номер"].isin(no_text)]

In [39]:
# df2.style.set_properties(subset=['текст'], **{'width': '300px'})

In [45]:
# df2[["номер", "текст"]].style.set_properties(subset=['текст'], **{'width': '400px'})

In [47]:
mapping = {2213:2209, 2214:2210, 2215:2211, 2216:2212, 2217:2213, 2218:2214, 2219:2215, 2220:2216, 2339:2332, 2340:2333, 2341:2334, 2342:2335, 2343:2343, 2344:2337, 2345:2338, 2346:2339, 2484:2475, 2912:2874, 3023:2978, 3152:3107, 3234:2189, 3239:3194, 3240:3195, 3241:3196, 3242:3197, 3243:3198, 3244:3199, 3245:3200, 3246:3201, 3406:3361, 3684:3639, 3685:3640, 3686:3641, 3687:3642, 3688:3543, 3689:3644, 3690:3645, 3691:3646, 3748:3703, 3867:3822, 4036:3988, 4037:3989, 4138:5944, 4230:5980, 4523:6051, 4582:5491, 4658:5632, 4664:5626}

In [50]:
result = []

for i in range(df.shape[0]):
    row = df.iloc[i]
    text_id = (clean(row["текст"])[:K], row["год"])
    if (text_id in text_map) or (row["номер"] not in mapping):
        continue
    names = []
    for name in ["собиратель", "собиратель2", "собиратель3", "собиратель4"]:
        if type(row[name]) == str:
            if "," in row[name]:
                names.extend(row[name].split(","))
            elif row[name].strip("???"):
                names.append(row[name])
    if set([n for n in names if n not in collectors_dict]):
        print("||", row["номер"], set([n for n in names if n not in collectors_dict]))
    for n in names:
        result.append((mapping[row["номер"]], collectors_dict[n]))

In [52]:
cur.executemany("INSERT INTO t_c (id_text, id_collector) VALUES (%s, %s)", result)
con.commit()

**Информанты фикс по маппингу текстов**

In [53]:
collectors_dict = dict(pd.read_sql_query("SELECT code, id FROM informators", con=con).values)

In [57]:
result = []

for i in range(df.shape[0]):
    row = df.iloc[i]
    text_id = (clean(row["текст"])[:K], row["год"])
    if (text_id in text_map) or (row["номер"] not in mapping):
        continue
    names = []
    for name in ["информант", "информант2", "информант3", "информант4"]:
        if type(row[name]) == str:
            if "," in row[name]:
                names.extend(row[name].split(","))
            elif row[name].strip("???"):
                names.append(row[name])
    if set([n for n in names if n not in collectors_dict]):
        print("||", row["номер"], set([n for n in names if n not in collectors_dict]))
    for n in names:
        result.append((mapping[row["номер"]], collectors_dict[n]))

In [58]:
cur.executemany("INSERT INTO t_i (id_text, id_informator) VALUES (%s, %s)", result)
con.commit()

## Видео пропавшие

In [6]:
df = pd.read_csv("/home/dkbrz/dumps/umk_videos.csv")

In [7]:
cur.executemany("INSERT INTO t_video (id_text, video) VALUES (%s,%s)", df.values.tolist())
con.commit()